# EMFAC - Extraction

### Imports

In [1]:
import requests
import pandas as pd
from datetime import datetime

# Local imports
from emfac_payloads import payload

In [2]:
%load_ext jupyter_black

### Parameters

In [3]:
SERVICE_API = "https://arb.ca.gov/emfac/handler/request_emfac2021.php"


headers = {
    "Host": "arb.ca.gov",
    "Origin": "https://arb.ca.gov",
    "Referer": "https://arb.ca.gov/emfac/emissions-inventory/4592b9779ca87f0b0d21346bbf923961ad4d3b24",
    "Sec-Ch-Ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
}

### Functions

In [15]:
def get_emfac_data(payload, initial_year=2015, final_year=None):
    """Get data from EMFAC API.
    and parse it into a DF
    """

    INITIAL_MODEL_YEAR = 1965
    if final_year is None:
        final_year = 2050

    years = list(range(initial_year, final_year + 1))
    model_years = list(range(INITIAL_MODEL_YEAR, final_year + 1))
    new_payload = payload.copy()
    new_payload["form"]["calendarYear"] = years
    new_payload["form"]["calendarYearStart"] = initial_year
    new_payload["form"]["calendarYearEnd"] = final_year
    new_payload["form"]["calendarYearStartSelected"] = initial_year
    new_payload["form"]["calendarYearEndSelected"] = final_year
    new_payload["form"]["modelYear"] = model_years
    new_payload["form"]["modelYearStart"] = INITIAL_MODEL_YEAR
    new_payload["form"]["modelYearEnd"] = final_year
    new_payload["form"]["modelYearStartSelected"] = INITIAL_MODEL_YEAR
    new_payload["form"]["modelYearEndSelected"] = final_year

    response = requests.post(SERVICE_API, json=new_payload, headers=headers)
    content = response.json()
    df = pd.DataFrame(content["output"])

    return df

### Pipeline Tests

In [16]:
df = get_emfac_data(payload, initial_year=2015, final_year=2050)

{'form': {'output': 'Emissions', 'regionType': 'Sub-Area', 'modelVersion': 'emfac2021', 'modelVersionNumber': {'label': 'v1.0.2', 'value': 'v102_byspeed'}, 'modelVersionNumberLabel': 'v1.0.2', 'modelVersionNumberValue': 'v102_byspeed', 'region': [{'id': '59', 'name': 'Los Angeles (SC)'}], 'calendarYear': [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050], 'season': 'Annual', 'scenario': 'exh', 'vehicleCategoryMode': 'emfac202x', 'vehicleCategory': ['LDA', 'LDT1', 'LDT2', 'MDV', 'MCY', 'MH', 'LHD1', 'LHD2', 'T6 Public Class 4', 'T6 Public Class 5', 'T6 Public Class 6', 'T6 Public Class 7', 'T6 Utility Class 5', 'T6 Utility Class 6', 'T6 Utility Class 7', 'T6 Instate Tractor Class 6', 'T6 Instate Delivery Class 4', 'T6 Instate Delivery Class 5', 'T6 Instate Delivery Class 6', 'T6 Instate Other Class 4', 'T6 Instate Other Cla

In [17]:
df

,No,Region,Calendar Year,Vehicle Category,Model Year,Speed,Fuel,Population,Total VMT,CVMT,...,CO_RUNEX,CO_IDLEX,CO_STREX,CO_TOTEX,SOx_RUNEX,SOx_IDLEX,SOx_STREX,SOx_TOTEX,NH3_RUNEX,Fuel Consumption
0,1,Los Angeles (SC),2015,All Other Buses,Aggregate,Aggregate,Diesel,1953.762385962299,24609566.48576078,24609566.48576078,...,52.97565244669009,2.2035095843535166,0,55.17916203104361,0.29087872098519235,0.0039378277830101295,0,0.29481654876820246,2.3174238871031076,2781.1519128048385
1,2,Los Angeles (SC),2015,All Other Buses,Aggregate,Aggregate,Natural Gas,159.1376130264676,2755693.919855186,2755693.919855186,...,8.253545636442539,0.2503832647850134,0,8.503928901227551,0,0,0,0,3.2198896091166596,327.7037422262372
2,3,Los Angeles (SC),2015,LDA,Aggregate,Aggregate,Gasoline,3825406.0401957175,48426254296.85113,48426254296.85113,...,95882.96906828054,0,38986.229135246496,134869.19820352702,173.50044205233922,0,5.637542906678407,179.13798495901764,1435.716562923126,1910766.9575570372
3,4,Los Angeles (SC),2015,LDA,Aggregate,Aggregate,Diesel,23292.5809435353,285995884.10141146,285995884.10141146,...,135.5136826785281,0,0,135.5136826785281,0.7858738860445201,0,0,0.7858738860445201,0.9772948634670717,7408.763881707742
4,5,Los Angeles (SC),2015,LDA,Aggregate,Aggregate,Electricity,21163.715632881816,243991744.6426801,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4582,4583,Los Angeles (SC),2050,T7IS,Aggregate,Aggregate,Gasoline,8.382095263434701,324753.8158949156,324753.8158949156,...,9.446769491550784,0,0.2911337451609585,9.737903236711743,0.005686982015567832,0,0.000025256981455085277,0.005712238997022917,0.01610908658682761,60.929330714963484
4583,4584,Los Angeles (SC),2050,T7IS,Aggregate,Aggregate,Electricity,8.505068163006207,402287.7048895692,0,...,0,0,0,0,0,0,0,0,0,0
4584,4585,Los Angeles (SC),2050,UBUS,Aggregate,Aggregate,Gasoline,234.663553253564,4455525.685503978,4455525.685503978,...,2.337982832015047,0,1.9020949572173926,4.24007778923244,0.027373615676058154,0,0.00011393062602896229,0.027487546302087117,0.22101187282382204,293.1946299963293
4585,4586,Los Angeles (SC),2050,UBUS,Aggregate,Aggregate,Electricity,7069.046363660655,242852091.12588802,0,...,0,0,0,0,0,0,0,0,0,0
